In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np           
import xml.etree.ElementTree as Et
from PIL import Image as im
import math 
from sklearn.linear_model import Ridge
from utils import *
import pandas as pd
import matplotlib.pyplot as plt



import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets,transforms,models
import matplotlib.pyplot as plt
from PIL import Image as im
import time
import copy
from tqdm import tqdm





# obj_class =[]
# for e,i in enumerate(os.listdir( annot_path)):
#     xml =  open(os.path.join(annot_path, i), "r")
#     tree = Et.parse(xml)
#     root = tree.getroot()
#     objects = root.findall("object")
#     obj_class = obj_class + [_object.find('name').text for _object in objects ]
#     obj_class = list(set(obj_class))
#     if len(obj_class) == 20:
#         break


obj_class = ['person', 
           'bird', 
           'cat', 
           'cow', 
           'dog',    # 5
           'horse', 
           'sheep', 
           'aeroplane', 
           'bicycle', 
           'boat',   # 10
           'bus', 
           'car', 
           'motorbike', 
           'train', 
           'bottle', 
           'chair',  # 16
           'diningtable', 
           'pottedplant', 
           'sofa', 
           'tvmonitor'
           ]

obj_class = ['background'] + obj_class




In [10]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [11]:
li=os.listdir('VOC2007_train_val/JPEGImages')
print('total data set:',len(li))

total data set: 5011


# VOC dataset을 selective search한 결과를 폴더에 저장하는 함수

In [12]:
from collections import defaultdict

annot_path='VOC2007_train_val/Annotations'      # annotaion data가 있는 경로
img_path='VOC2007_train_val/JPEGImages'      # 이미지 데이터가 있는 경로
count=0
count_back=0

for e,i in enumerate(os.listdir(annot_path)):   # annotatoin 파일을 읽어서 정보를 저장 파일 하나에 여러개의 물체의 정보가 있을수 있음
    filename = i.split(".")[0]
    img_file = i.split(".")[0]+".jpg"
    if e%100 == 0:
        print('Success:', e)
    image = cv2.imread(os.path.join(img_path,img_file))
    xml =  open(os.path.join(annot_path, i), "r")
    tree = Et.parse(xml)
    root = tree.getroot()
    objects = root.findall("object")
    imout = image.copy()
    
    
    bb_values=[]
    for _object in objects:               # ground truth 정보 저장
        bndbox = _object.find('bndbox')   # object -> bndbox 객체 반환
        obj_name = _object.find('name').text   
        xmin = int(bndbox.find('xmin').text)
        xmax = int(bndbox.find('xmax').text)
        ymin = int(bndbox.find('ymin').text)
        ymax = int(bndbox.find('ymax').text)
        bb_values.append({"name": obj_name,"xmin":xmin,"xmax":xmax,"ymin":ymin,"ymax":ymax})
        


    ss.setBaseImage(image)                    # 위 annotation파일과 대응되는 image파일 한장에서 
    ss.switchToSelectiveSearchFast()          # selective search 알고리즘으로 객체가 있을법한 영역 검출
    ssresults = ss.process()                  # e번쨰 이미지에서 (266(selective search로 찾은 박스),4)
    imout = image.copy()
 

    

    for e1,result in enumerate(ssresults):
        if e1 < 2000:
            for gtval in bb_values:
                label=obj_class.index(gtval['name'])# 0~20, 0 -> background
                
                x,y,w,h = result
                iou = get_iou(gtval,{"xmin":x,"xmax":x+w,"ymin":y,"ymax":y+h})   # ground truth에 존재하는 객체의 박스와 selective search 로 찾은 박스의 IOU를 계산
                
                path='data/'+str(label)
                if os.path.exists(path)==False:
                    os.makedirs(path)

                if iou > 0.50:
                    timage = imout[y:y+h,x:x+w]
                    resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)  # 선택된 영역 resize
                    cv2.imwrite('data/{}/{}.jpg'.format(label,count),resized)
                else:
                    if count_back>100000:
                        continue
                    else:
                        timage = imout[y:y+h,x:x+w]
                        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                        cv2.imwrite('data/0/{}.jpg'.format(count_back),resized)
                        count_back+=1
                count+=1
       

    

Success: 0
Success: 100
Success: 200
Success: 300
Success: 400
Success: 500
Success: 600
Success: 700
Success: 800
Success: 900
Success: 1000
Success: 1100
Success: 1200
Success: 1300
Success: 1400
Success: 1500
Success: 1600
Success: 1700
Success: 1800
Success: 1900
Success: 2000
Success: 2100
Success: 2200
Success: 2300
Success: 2400
Success: 2500
Success: 2600
Success: 2700
Success: 2800
Success: 2900
Success: 3000
Success: 3100
Success: 3200
Success: 3300
Success: 3400
Success: 3500
Success: 3600
Success: 3700
Success: 3800
Success: 3900
Success: 4000
Success: 4100
Success: 4200
Success: 4300
Success: 4400
Success: 4500
Success: 4600
Success: 4700
Success: 4800
Success: 4900
Success: 5000
